In [ ]:
import pandas as pd
import altair as alt
import warnings
import vl_convert as vlc
import cairosvg
warnings.filterwarnings('ignore')
alt.data_transformers.disable_max_rows()

In [ ]:
def read_results(prefix, expt_name, since=0):
    results_file = f"results/infer_bench_perf/{prefix}_{expt_name}.csv"
    if expt_name.startswith("High"):
        usage = "3. High"
    if expt_name == "Medium":
        usage = "2. Medium"
    if expt_name == "Low":
        usage = "1. Low" if prefix == "pre" else "4. Low"
    columns = ["since", "latency", "exec_mode"]
    results = pd.read_csv(results_file, names=columns)
    results["latency"] = results.latency * 1000.0 # Convert to ms.
    results["since"] = results["since"] / 1000.0 + since
    results["Usage"] = [usage for _ in range(0, len(results))]
    p99 = results.latency.quantile(0.99)
    results = results[results.latency < p99]
    return results


res1 = read_results("pre", "Low", 0)
low_avg = res1.latency.mean()
res2 = read_results("pre", "Medium", res1.since.max())
medium_avg = res2.latency.mean()
res3 = read_results("pre", "High(10)", res2.since.max())
high_avg = res3.latency.mean()
pre_res = pd.concat([res1, res2, res3])
res4 = read_results("post", "Low", res3.since.max())
res = pd.concat([res1, res2, res3, res4])


# print("Latencies")
# chart1.display()

# expt_dir = "figures/inference"
# svg_str = vlc.vegalite_to_svg(chart1.to_json())
# cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/latencies.pdf")

res["since"] = pd.to_timedelta(res.since, unit='s')
res = res[["since", "latency"]].groupby(pd.Grouper(key="since", freq="1min")).median()
res["since"] = res.index.total_seconds() / 60 + 1

chart1 = alt.Chart(res).mark_line().encode(
    x = alt.X('since', title='Time since Start'),
    y = alt.Y('latency', title='End-to-end lantency (ms)'),
).properties(
    # title = "Read performance distribution",
    width=300,
    height=300
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

chart1

In [ ]:
# Avg cost per month.
def comparison_nums(avg_latency, median_latency, mode):
    if mode == "Low":
        reqs_per_sec = 1
    if mode == "Medium":
        reqs_per_sec = 50
    if mode == "High":
        reqs_per_sec = 200
    # Reqs in an month.
    num_hours = 24 * 30
    total_reqs = 3600.0 * reqs_per_sec * num_hours
    print(total_reqs)
    pure_lambda_latency = low_median # Same for all other modes.
    print(pure_lambda_latency)
    pure_lambda_cost = total_reqs * ((low_avg - 15.0) * 4 * 0.0000000167 + 0.2 / 1e6) # 4GB of mem. 
    pure_lambda_user_cost = total_reqs * low_avg * 0.5 * 0.0000000167
    visc_latency = median_latency
    if mode == "Low":
        visc_cost = pure_lambda_cost
    if mode == "Medium":
        visc_cost = num_hours * 0.015 * 2.0 * 2.0 # 2vcpus (x2 for excess sizing).
    if mode == "High":
        visc_cost  = num_hours * 9 * (0.015 * 2.0 * 2.0) # Like above, but with 3 instances.
    visc_user_cost = total_reqs * avg_latency * 0.5 * 0.0000000167
    
    systems = []
    modes = []
    latencies = []
    compute_costs = []
    overall_costs = []
    if mode == "Low":
        mode = "1. Low"
    if mode == "Medium":
        mode = "2. Medium"
    if mode == "High":
        mode = "3. High"
    systems.append("Pure Lambda")
    modes.append(mode)
    latencies.append(pure_lambda_latency)
    compute_costs.append(pure_lambda_cost)
    overall_costs.append(pure_lambda_cost + pure_lambda_user_cost)
    systems.append("VISC Inference")
    modes.append(mode)
    latencies.append(visc_latency)
    compute_costs.append(visc_cost)
    overall_costs.append(visc_cost + visc_user_cost)
    return pd.DataFrame({
        "systems": systems,
        "modes": modes,
        "latencies": latencies,
        "compute_costs": compute_costs,
        "overall_costs": overall_costs,
    })



low_median = res1.latency.quantile(0.5)
medium_median = res2.latency.quantile(0.5)
high_median = res3.latency.quantile(0.5)
cres1 = comparison_nums(low_avg, low_median, "Low")
cres2 = comparison_nums(medium_avg, medium_median, "Medium")
cres3 = comparison_nums(high_avg, high_median, "High")
cres = pd.concat([cres1, cres2, cres3])

chart1 = alt.Chart(data=cres).mark_bar().encode(
    x=alt.X("systems", title=""),
    y=alt.Y("compute_costs", title="Monthly Cost"),
    column=alt.Column("modes", title="Utilization"),
    color=alt.Color("systems", legend=None)
).properties(
    width=200,
    height=100,
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

chart2 = alt.Chart(data=cres).mark_bar().encode(
    x=alt.X("systems", title=""),
    y=alt.Y("overall_costs", title="Monthly Cost"),
    column=alt.Column("modes", title="Utilization"),
    color=alt.Color("systems", legend=None)
).properties(
    width=200,
    height=100,
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

chart3 = alt.Chart(data=cres).mark_bar().encode(
    x=alt.X("systems", title=""),
    y=alt.Y("latencies", title="Latency"),
    column=alt.Column("modes", title="Utilization"),
    color=alt.Color("systems", legend=None)
).properties(
    width=200,
    height=100,
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

print("Compute Costs")
chart1.display()
print("Overall Costs")
chart2.display()
print("Latencies")
chart3.display()

expt_dir = "figures/inference"
svg_str = vlc.vegalite_to_svg(chart1.to_json())
cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/compute_cost_comparison.pdf")
svg_str = vlc.vegalite_to_svg(chart2.to_json())
cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/overall_cost_comparison.pdf")
svg_str = vlc.vegalite_to_svg(chart3.to_json())
cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/latency_comparison.pdf")

cres

In [ ]:
52.790602 * (1 + 0.1585189538092405)

In [ ]:
def read_results(prefix, expt_name, since=0):
    results_file = f"results/microactor_bench/{prefix}_{expt_name}.csv"
    if expt_name.startswith("High"):
        usage = "3. High"
    if expt_name == "Medium":
        usage = "2. Medium"
    if expt_name == "Low":
        usage = "1. Low" if prefix == "pre" else "4. Low"
    columns = ["since", "latency", "operation"]
    results = pd.read_csv(results_file, names=columns)
    results["latency"] = results.latency * 1000.0 # Convert to ms.
    results["since"] = results["since"] / 1000.0 + since
    results["Usage"] = [usage for _ in range(0, len(results))]
    p99 = results.latency.quantile(0.99)
    results = results[results.latency < p99]
    return results

# Compute results.
res1 = read_results("pre", "Low", 0)
low_avg = 2 * res1.latency.mean() # One retrieve and one increment request.
res2 = read_results("pre", "Medium", res1.since.max())
medium_avg = 2 * res2.latency.mean()
res3 = read_results("pre", "High(10)", res2.since.max())
high_avg = 2 * res3.latency.mean()
pre_res = pd.concat([res1, res2, res3])
res4 = read_results("post", "Low", res3.since.max())
res = pd.concat([res1, res2, res3, res4])

chart1 = alt.Chart(res[res.operation == "Retrieve"]).mark_boxplot(extent="min-max", size=50).encode(
    x = alt.X('Usage', title='Utilization'),
    y = alt.Y('latency', scale=alt.Scale(type="symlog"), axis=alt.Axis(values=[0, 1, 10, 100, 1000]), title='End-to-end lantency (ms)'),
).properties(
    # title = "Read performance distribution",
    width=300,
    height=300
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

chart2 = alt.Chart(res[res.operation == "Increment"]).mark_boxplot(extent="min-max", size=50).encode(
    x = alt.X('Usage', title='Utilization'),
    y = alt.Y('latency', scale=alt.Scale(type="symlog"), axis=alt.Axis(values=[0, 1, 10, 100, 1000]), title='End-to-end lantency (ms)'),
).properties(
    # title = "Read performance distribution",
    width=300,
    height=300
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

print("Retrieve Latencies")
chart1.display()
print("Increment Latencies")
chart2.display()


expt_dir = "figures/actor/"
svg_str = vlc.vegalite_to_svg(chart1.to_json())
cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/retrieve_latencies.pdf")
svg_str = vlc.vegalite_to_svg(chart2.to_json())
cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/increment_latencies.pdf")

In [ ]:

# Avg cost per month.
def comparison_nums(avg_latency, median_latency, mode):
    if mode == "Low":
        reqs_per_sec = 1
    if mode == "Medium":
        reqs_per_sec = 100
    if mode == "High":
        reqs_per_sec = 1000
    # Reqs in an month.
    num_hours = 24 * 30
    total_reqs = 3600.0 * reqs_per_sec * num_hours
    print(total_reqs)
    pure_lambda_latency = low_median # Same for all other modes.
    print(pure_lambda_latency)
    pure_lambda_cost = total_reqs * ((low_avg - 10.0) * 4 * 0.0000000167 + 0.2 / 1e6) # 4GB of mem.
    pure_lambda_user_cost = total_reqs * low_avg * 0.5 * 0.0000000167
    pure_ecs_latency = medium_median
    pure_ecs_cost = num_hours * 0.015 * 2.0 # 2vcpus.
    pure_ecs_user_cost = total_reqs * medium_avg * 0.5 * 0.0000000167
    visc_latency = median_latency
    if mode == "Low":
        visc_cost = pure_lambda_cost
    if mode == "Medium":
        visc_cost = pure_ecs_cost
    if mode == "High":
        visc_cost  = num_hours * 0.015 * (2.0 + 6*0.25) # 2vcpus and 6 times 0.25vcpus.
    visc_user_cost = total_reqs * avg_latency * 0.5 * 0.0000000167
    
    systems = []
    modes = []
    latencies = []
    compute_costs = []
    overall_costs = []
    if mode == "Low":
        mode = "1. Low"
    if mode == "Medium":
        mode = "2. Medium"
    if mode == "High":
        mode = "3. High"
    if "High" not in mode:
        systems.append("Lambda")
        modes.append(mode)
        latencies.append(pure_lambda_latency)
        compute_costs.append(pure_lambda_cost)
        overall_costs.append(pure_lambda_cost + pure_lambda_user_cost)
    systems.append("Container")
    modes.append(mode)
    latencies.append(pure_ecs_latency)
    compute_costs.append(pure_ecs_cost)
    overall_costs.append(pure_ecs_cost + pure_ecs_user_cost)
    systems.append("VISC")
    modes.append(mode)
    latencies.append(visc_latency)
    compute_costs.append(visc_cost)
    overall_costs.append(visc_cost + visc_user_cost)
    return pd.DataFrame({
        "systems": systems,
        "modes": modes,
        "latencies": latencies,
        "compute_costs": compute_costs,
        "overall_costs": overall_costs,
    })

# Compute medians.
low_median = res1[res1["operation"] == "Retrieve"].latency.quantile(0.5) + res1[res1["operation"] == "Increment"].latency.quantile(0.5)
medium_median = res2[res2["operation"] == "Retrieve"].latency.quantile(0.5) + res2[res2["operation"] == "Increment"].latency.quantile(0.5)
high_median = res3[res3["operation"] == "Retrieve"].latency.quantile(0.5) + res3[res3["operation"] == "Increment"].latency.quantile(0.5)
cres1 = comparison_nums(low_avg, low_median, "Low")
cres2 = comparison_nums(medium_avg, medium_median, "Medium")
cres3 = comparison_nums(high_avg, high_median, "High")
cres = pd.concat([cres1, cres2, cres3])

chart1 = alt.Chart(data=cres).mark_bar().encode(
    x=alt.X("systems", title=""),
    y=alt.Y("compute_costs", title="Monthly Cost"),
    column=alt.Column("modes", title="Utilization"),
    color=alt.Color("systems", legend=None)
).properties(
    width=200,
    height=100,
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

chart2 = alt.Chart(data=cres).mark_bar().encode(
    x=alt.X("systems", title=""),
    y=alt.Y("overall_costs", title="Monthly Cost"),
    column=alt.Column("modes", title="Utilization"),
    color=alt.Color("systems", legend=None)
).properties(
    width=200,
    height=100,
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

chart3 = alt.Chart(data=cres).mark_bar().encode(
    x=alt.X("systems", title=""),
    y=alt.Y("latencies", title="Latency"),
    column=alt.Column("modes", title="Utilization"),
    color=alt.Color("systems", legend=None)
).properties(
    width=200,
    height=100,
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

print("Compute Costs")
chart1.display()
print("Overall Costs")
chart2.display()
print("Latencies")
chart3.display()

expt_dir = "figures/actor"
svg_str = vlc.vegalite_to_svg(chart1.to_json())
cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/compute_cost_comparison.pdf")
svg_str = vlc.vegalite_to_svg(chart2.to_json())
cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/overall_cost_comparison.pdf")
svg_str = vlc.vegalite_to_svg(chart3.to_json())
cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/latency_comparison.pdf")

cres

In [ ]:
260.138862/123.920105

In [ ]:
8 / 2.32

# KV Bench

In [ ]:
def read_results(prefix, expt_name, since=0):
    results_file = f"results/kv_bench_cost_only/{prefix}_{expt_name}.csv"
    if expt_name.startswith("High"):
        usage = "3. High"
    if expt_name == "Medium":
        usage = "2. Medium"
    if expt_name == "Low":
        usage = "1. Low" if prefix == "pre" else "4. Low"
    columns = ["since", "latency", "exec_mode"]
    results = pd.read_csv(results_file, names=columns)
    results["latency"] = results.latency * 1000.0 # Convert to ms.
    results["since"] = results["since"] / 1000.0 + since
    results["Usage"] = [usage for _ in range(0, len(results))]
    p99 = results.latency.quantile(0.99)
    results = results[results.latency < p99]
    return results


res1 = read_results("kvpre", "Low", 0)
low_avg = res1.latency.mean()
res2 = read_results("kvpre", "Medium", res1.since.max())
medium_avg = res2.latency.mean()
res3 = read_results("kvpre", "High(10)", res2.since.max())
high_avg = res3.latency.mean()
pre_res = pd.concat([res1, res2, res3])
res4 = read_results("kvpost", "Low", res3.since.max())
res = pd.concat([res1, res2, res3, res4])
res = res[res["exec_mode"] == "Read"]

# print("Latencies")
# chart1.display()

# expt_dir = "figures/inference"
# svg_str = vlc.vegalite_to_svg(chart1.to_json())
# cairosvg.svg2pdf(bytestring=svg_str, write_to=f"{expt_dir}/latencies.pdf")

res["since"] = pd.to_timedelta(res.since, unit='s')
res = res[["since", "latency"]].groupby(pd.Grouper(key="since", freq="1min")).median()
res["since"] = res.index.total_seconds() / 60 + 1

chart1 = alt.Chart(res).mark_line().encode(
    x = alt.X('since', title='Time since Start'),
    y = alt.Y('latency', title='End-to-end lantency (ms)'),
).properties(
    # title = "Read performance distribution",
    width=300,
    height=300
).configure_axis(
    titleFontSize=12,
    labelFontSize=12
).configure_legend(
    titleFontSize=12,
    labelFontSize=12,
)

chart1

In [ ]:
res.iloc[160:180]